<a href="https://colab.research.google.com/github/aleks-haksly/Postypashka/blob/main/DA/02%20HW_lab/HW_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import pandas as pd

# Подготовка данных для анализа

1. Скачаем реестр анализируемых компаний
2. Прочитаем Excel файл и создадим из него DataFrame. Считывать будем только колонки, которые будут использоваться в дальнейших расчетах показателя
[EBITDA](https://xn--80aapampemcchfmo7a3c9ehj.xn--p1ai/news/chto-takoe-ebitda/)


In [ ]:
!wget "https://github.com/aleks-haksly/Postypashka/raw/main/DA/02%20HW/%D0%A0%D0%B5%D0%B5%D1%81%D1%82%D1%80.xlsx" -q

In [ ]:
df_reestr = pd.read_excel("/content/Реестр.xlsx", header = 2, usecols=[1, 2, 3, 5, 6]) # nrows=1000

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


3. Отфильтруем DataFrame и сохраним его копию, содержащую данные отвечающие всем условиям:
* Юридическое лицо
* Среднее предприятие либо Малое предприятие
* Основной вид деятельности - "41.20 Строительство жилых и нежилых зданий"

In [ ]:
df_reestr = df_reestr.query("(`Категория` in ('Среднее предприятие', 'Малое предприятие')) & \
                             (`Основной вид деятельности` == '41.20 Строительство жилых и нежилых зданий') &\
                             (`Тип субъекта` == 'Юридическое лицо')")[['Наименование / ФИО', 'ИНН']]

In [ ]:
df_reestr.to_excel("reestr_filtered.xlsx")

In [7]:
!wget "https://github.com/aleks-haksly/Postypashka/raw/main/DA/02%20HW_lab/reestr_filtered.xlsx" -q

In [10]:
df_reestr = pd.read_excel("reestr_filtered.xlsx", index_col=[0])

# Парсинг данных

In [14]:
df_reestr.head()

,Наименование / ФИО,ИНН
19,"""КОРПОРАЦИЯ ВИТ"" (ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕ...",5038038838
51,"""ХОЗРАСЧЕТНАЯ СТРОИТЕЛЬНО-ТЕХНОЛОГИЧЕСКАЯ ФИРМ...",5027006369
4652,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""777""",1414006922
4667,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АГРОТЕХИМПОРТ""",3327332190
4673,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АКС""",7816061297


In [12]:
headers ={'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.28 Safari/537.36'}
session = requests.Session()
session.headers = headers

In [13]:
res = session.get(f"https://bo.nalog.ru/advanced-search/organizations/search?query=5038038838&page=0").json()
org_id = res["content"][0]["id"]


ConnectTimeout: HTTPSConnectionPool(host='bo.nalog.ru', port=443): Max retries exceeded with url: /advanced-search/organizations/search?query=5038038838&page=0 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7b9cc4a58790>, 'Connection to bo.nalog.ru timed out. (connect timeout=None)'))